In [1]:
import glob as glob
import pandas as pd

redact_germline = True

# Process retrospective cohorts
Both the illustrate notebook and calculations notebook will require the outputs from both MOAlmanac and PHIAL to be processed. In particular, we clean up the outputs and annotate the two so that we know how the other software annotated the same molecular feature.

In [2]:
def load_handles_simple(handles):
    list_ = []
    for handle in handles:
        patient_id = handle.split('/')[-1].split('_complete_muts_indels_scna_detailed.txt')[0]
        df = pd.read_csv(handle, sep='\t')
        df['patient_id'] = patient_id
        list_.append(df)
    return pd.concat(list_, ignore_index=True)
    
def load_handles(handles):
    list_ = []
    for handle in handles:
        list_.append(pd.read_csv(handle, sep='\t'))
    df = pd.concat(list_, ignore_index=True)
    return df

perry_phial_handles = glob.glob('2014-Perry/data/phial/*.txt')
robinson_phial_handles = glob.glob('2015-Robinson/data/phial/*.txt')
tcga_phial_handles = glob.glob('2016-TCGA/data/phial/*.txt')
vanallen_phial_handles = glob.glob('2015-VanAllen/data/phial/*.txt')

perry_handles = glob.glob('2014-Perry/data/almanac/*/*.actionable.txt')
robinson_handles = glob.glob('2015-Robinson/data/almanac/*/*.actionable.txt')
tcga_handles = glob.glob('2016-TCGA/data/almanac/*/*.actionable.txt')
vanallen_handles = glob.glob('2015-VanAllen/data/almanac/*/*.actionable.txt')

perry_phial = load_handles_simple(perry_phial_handles)
robinson_phial = load_handles_simple(robinson_phial_handles)
tcga_phial = load_handles_simple(tcga_phial_handles)
vanallen_phial = load_handles_simple(vanallen_phial_handles)

perry_almanac = load_handles(perry_handles)
robinson_almanac = load_handles(robinson_handles)
tcga_almanac = load_handles(tcga_handles)
vanallen_almanac = load_handles(vanallen_handles)

perry_almanac['cohort'] = 'OS'
perry_phial['cohort'] = 'OS'
robinson_almanac['cohort'] = 'SU2C'
robinson_phial['cohort'] = 'SU2C'
tcga_almanac['cohort'] = 'KIRP'
tcga_phial['cohort'] = 'KIRP'
vanallen_almanac['cohort'] = 'Melanoma'
vanallen_phial['cohort'] = 'Melanoma'

phial = pd.concat([perry_phial, robinson_phial, tcga_phial, vanallen_phial], ignore_index=True)
almanac = pd.concat([perry_almanac, robinson_almanac, tcga_almanac, vanallen_almanac], ignore_index=True)
if redact_germline:
    almanac = almanac[~almanac['feature_type'].eq('Germline Variant')]

phial = phial[~phial['Score_bin'].eq('Filtered Calls')]
phial = phial[~phial['Variant_Classification'].eq('lincRNA')]

target = pd.read_excel('../knowledge-bases/target/almanac-comparison.xlsx')
phial = phial[phial['Gene'].isin(target['Gene'])]

In [3]:
def annotate_feature_string_phial(dataframe):
    dataframe['Alteration'] = dataframe['Alteration'].replace({'Deleted': 'Deletion', 'Amplified': 'Amplification'})
    dataframe['feature_string'] = ''
    dataframe['feature_string'] = (dataframe['Gene'].fillna('') + '.' + dataframe['Alteration'].fillna(''))
    idx_class = dataframe['Variant_Classification'].isin(['Splice_Site', 'Start_Codon_Del'])
    dataframe.loc[idx_class, 'feature_string'] += dataframe.loc[idx_class, 'Variant_Classification']
    return dataframe

def annotate_feature_string_almanac(dataframe):
    dataframe['feature_string'] = ''
    idx_mutation = dataframe[dataframe['feature_type'] == 'Somatic Variant'].index
    idx_copynumber = dataframe[dataframe['feature_type'] == 'Copy Number'].index
    idx_fusion = almanac[almanac['feature_type'].eq('Rearrangement')].index
    idx_germline = almanac[almanac['feature_type'].eq('Germline Variant')].index
    idx_wgd = almanac[almanac['feature_type'].eq('Aneuploidy')].index
    idx_tmb = almanac[almanac['feature_type'].eq('Mutational Burden')].index
    idx_signature = almanac[almanac['feature_type'].eq('Mutational Signature')].index
    
    dataframe.loc[idx_mutation, 'feature_string'] = (dataframe.loc[idx_mutation, 'feature'].fillna('') + '.' + 
                                                     dataframe.loc[idx_mutation, 'alteration'].fillna(''))
    dataframe.loc[idx_copynumber, 'feature_string'] = (dataframe.loc[idx_copynumber, 'feature'].fillna('') + '.' + 
                                                       dataframe.loc[idx_copynumber, 'alteration_type'].fillna(''))
    dataframe.loc[idx_fusion, 'feature_string'] = dataframe.loc[idx_fusion, 'alteration']
    dataframe.loc[idx_germline, 'feature_string'] = dataframe.loc[idx_germline, 'feature'].fillna('') + '.' + dataframe.loc[idx_germline, 'alteration'].fillna('') + '.Germline'
    dataframe.loc[idx_wgd, 'feature_string'] = dataframe.loc[idx_wgd, 'feature'].fillna('')
    dataframe.loc[idx_tmb, 'feature_string'] = dataframe.loc[idx_tmb, 'feature'].fillna('')
    dataframe.loc[idx_signature, 'feature_string'] = dataframe.loc[idx_signature, 'feature'].fillna('')
    
    idx_class = dataframe['alteration_type'].isin(['Splice Site', 'Start_Codon_Del'])
    dataframe.loc[idx_class, 'feature_string'] += dataframe.loc[idx_class, 'alteration_type'].str.replace(' ', '_')
    return dataframe

phial = annotate_feature_string_phial(phial)
almanac = annotate_feature_string_almanac(almanac)

phial['patient_feature_string'] = phial['patient_id'].fillna('') + '.' + phial['feature_string'].fillna('')
almanac['patient_feature_string'] = almanac['patient_id'].fillna('') + '.' + almanac['feature_string'].fillna('')

phial = phial[~phial['feature_string'].eq('')]
almanac = almanac[~almanac['feature_string'].eq('')]

In [4]:
# Removing double counted fusions. 
# This is a feature, not a bug, for reviewing individual fusions; however,
# it results in double counting for aggregated reporting. 
almanac.drop_duplicates(['patient_feature_string'], keep='first', inplace=True)

In [5]:
almanac['feature_type'].value_counts()

Somatic Variant         1299
Copy Number             1057
Mutational Signature     620
Rearrangement            255
Aneuploidy               180
Mutational Burden         48
Name: feature_type, dtype: int64

In [6]:
# The bin "Investigate Biological Significance" is used for CNAs that score within
# a TARGET gene but the directionality does not match. Since there are only 95 of them 
# I am going to label them as "Investigate Actionability". There is not an
# exact map to Almanac bins because PHIAL/TARGET doesn't check if this has been catalogued
# as a CNA, just the direction / alteration_type. Specifically, PHIAL will ask,
# "does TARGET have 'Amplification' catalogued for this gene?"
# Whereas MOAlmanac asks, "Okay, is this a catalogued gene? Is this a catalogued CNA? 
# Is this a catalogued Amplification?"

phial['Score_bin'] = phial['Score_bin'].replace({'Investigate Biological Significance': 'Investigate Actionability'})

# High priority is given to somatic variants whose gene appears in PHIAL but does not match any further
# These are most similar to the Biologically Relevant category
phial['Score_bin'] = phial['Score_bin'].replace({'High Priority': 'Biologically Relevant'})

In [7]:
all_patients = pd.Index(almanac['patient_id'].drop_duplicates().sort_values())
all_features = (
    pd.Index(almanac['feature_string'].drop_duplicates().sort_values())
    .union(
        pd.Index(phial['feature_string'].drop_duplicates().sort_values())
    ).drop_duplicates().tolist()
)

all_considered = pd.DataFrame(columns=['almanac_bin', 'phial_bin'],
                             index=pd.MultiIndex.from_product([all_patients, all_features]))

almanac_indexed = almanac.loc[:, ['patient_id', 'feature_string', 'score_bin']].drop_duplicates(['patient_id', 'feature_string']).set_index(['patient_id', 'feature_string'])
phial_indexed = phial.loc[:, ['patient_id', 'feature_string', 'Score_bin']].drop_duplicates(['patient_id', 'feature_string']).set_index(['patient_id', 'feature_string'])

all_considered.loc[almanac_indexed.index, 'almanac_bin'] = (
    almanac_indexed.loc[almanac_indexed.index, 'score_bin']
)

all_considered.loc[phial_indexed.index, 'phial_bin'] = (
    phial_indexed.loc[phial_indexed.index, 'Score_bin']
)

In [8]:
all_considered['sum'] = all_considered.fillna(False).astype(bool).sum(axis=1)
df = all_considered[all_considered['sum'].gt(0)]
df = df[~df['almanac_bin'].isnull() | df['phial_bin'].isin(['Actionable', 'High Priority', 'Investigate Actionability', 'Investigate Biological Significance', 'Biologically Relevant'])]
df = df.reset_index().rename(columns={'level_1': 'feature_string'})

df.loc[~df['phial_bin'].isin(['Actionable', 'Investigate Actionability', 'Biologically Relevant']), 'phial_bin'] = pd.NA
df['almanac_bin'] = df['almanac_bin'].fillna(pd.NA)
df['phial_bin'] = df['phial_bin'].str.replace('Actionable', 'Putatively Actionable')

In [9]:
df['patient_id'].value_counts()

MEL-IPI_Pat138               104
MEL-IPI_Pat110                97
MO_1012                       85
MEL-IPI_Pat28                 82
MEL-IPI_Pat58                 60
                            ... 
Osteosarcoma-BSP391            1
KIRP-B1-A47O                   1
KIRP-A4-A772                   1
Osteosarcoma-SJDOSTEOS014      1
MEL-IPI_Pat121                 1
Name: patient_id, Length: 417, dtype: int64

In [10]:
df.loc[df['patient_id'].str.contains('MEL'), 'cohort'] = 'MEL'
df.loc[df['patient_id'].str.contains('Osteosarcoma'), 'cohort'] = 'OS'
df.loc[df['patient_id'].str.contains('KIRP'), 'cohort'] = 'KIRP'
df.loc[df['cohort'].isnull(), 'cohort'] = 'SU2C'

In [11]:
df.tail()

,patient_id,feature_string,almanac_bin,phial_bin,sum,cohort
4024,TP_2064,COSMIC Signature 1,Biologically Relevant,<NA>,1,SU2C
4025,TP_2064,COSMIC Signature 6,Biologically Relevant,<NA>,1,SU2C
4026,TP_2064,MPL.p.A134T,Investigate Actionability,Biologically Relevant,2,SU2C
4027,TP_2064,TP53.p.H179R,Investigate Actionability,Investigate Actionability,2,SU2C
4028,TP_2064,Whole genome doubling,Investigate Actionability,<NA>,1,SU2C


In [12]:
df = df.merge(almanac.loc[:, ['sensitive_predictive_implication', 'patient_id', 'feature_string']],
        on=['patient_id', 'feature_string'], how='left')
df = df.merge(almanac.loc[:, ['resistance_predictive_implication', 'patient_id', 'feature_string']],
        on=['patient_id', 'feature_string'], how='left')
df = df.merge(almanac.loc[:, ['prognostic_predictive_implication', 'patient_id', 'feature_string']],
        on=['patient_id', 'feature_string'], how='left')
df = df.merge(almanac.loc[:, ['clinvar', 'patient_id', 'feature_string']],
        on=['patient_id', 'feature_string'], how='left')

In [13]:
df['feature_type'] = ''
df.loc[df['feature_string'].str.contains('Amplification'), 'feature_type'] = 'Somatic copy number'
df.loc[df['feature_string'].str.contains('Deletion'), 'feature_type'] = 'Somatic copy number'
df.loc[df['feature_string'].str.contains('--'), 'feature_type'] = 'Rearrangement'
df.loc[df['feature_string'].str.contains('Germline'), 'feature_type'] = 'Germline'
df.loc[df['feature_string'].eq('Whole genome doubling'), 'feature_type'] = 'Aneuploidy'
df.loc[df['feature_string'].eq('High Mutational Burden'), 'feature_type'] = 'Tumor mutational burden'
df.loc[df['feature_string'].str.contains('COSMIC Signature'), 'feature_type'] = 'Mutational signature'
df.loc[df['feature_type'].eq(''), 'feature_type'] = 'Somatic variant'

df['feature_string'].str.replace('.', ' ').str.replace('p ', 'p.')

0         ASXL1 Amplification
1         AURKA Amplification
2         COSMIC Signature 12
3          COSMIC Signature 4
4          TPX2 Amplification
                ...          
4024       COSMIC Signature 1
4025       COSMIC Signature 6
4026              MPL p.A134T
4027             TP53 p.H179R
4028    Whole genome doubling
Name: feature_string, Length: 4029, dtype: object

In [14]:
df['almanac_bin'].value_counts()

Biologically Relevant        1531
Investigate Actionability    1365
Putatively Actionable         563
Name: almanac_bin, dtype: int64

In [15]:
df['phial_bin'].value_counts()

Biologically Relevant        924
Investigate Actionability    657
Putatively Actionable         81
Name: phial_bin, dtype: int64

In [16]:
df['feature_type'].value_counts()

Somatic variant            1686
Somatic copy number        1240
Mutational signature        620
Rearrangement               255
Aneuploidy                  180
Tumor mutational burden      48
Name: feature_type, dtype: int64

In [17]:
df['feature_str_simple'] = ''
idx_direct = df['feature_type'].isin(['Mutational signature', 'Rearrangement'])
idx_wgd = df['feature_type'].eq('Aneuploidy')
idx_tmb = df['feature_type'].eq('Tumor mutational burden')
idx_germline = df['feature_type'].eq('Germline')
idx_cn = df['feature_type'].eq('Somatic copy number')
idx_som = df['feature_type'].eq('Somatic variant')

df.loc[idx_direct, 'feature_str_simple'] = df.loc[idx_direct, 'feature_string']
df.loc[idx_wgd, 'feature_str_simple'] = 'WGD'
df.loc[idx_tmb, 'feature_str_simple'] = 'TMB'
df.loc[idx_germline, 'feature_str_simple'] = (df.loc[idx_germline, 'feature_string']
                                              .str.split('.').str[0] 
                                              .add(' Germline'))
df.loc[idx_som, 'feature_str_simple'] = df.loc[idx_som, 'feature_string'].str.split('.').str[0]
df.loc[idx_cn, 'feature_str_simple'] = (df.loc[idx_cn, 'feature_string']
                                        .str.replace('Amplification', 'Amp')
                                        .str.replace('Deletion', 'Del')
                                        .str.replace('.', ' ')
                                       )


In [18]:
df['almanac_bin'].value_counts()

Biologically Relevant        1531
Investigate Actionability    1365
Putatively Actionable         563
Name: almanac_bin, dtype: int64

In [19]:
df['phial_bin'].value_counts()

Biologically Relevant        924
Investigate Actionability    657
Putatively Actionable         81
Name: phial_bin, dtype: int64

In [20]:
df.to_csv('retrospective.actionability.txt', sep='\t', index=False)

In [21]:
germline = almanac[almanac['feature_type'].eq('Germline Variant') & ~almanac['score_bin'].isin(['Biologically Relevant'])]
germline[germline['alteration_type'].eq('Missense')]['exac_common'].value_counts()

Series([], Name: exac_common, dtype: int64)

## Construct matrix of feature types x samples
0 = Wild type or biologically relevant for feature type  
1 = Molecular Oncology Almanac only Investigate Actionability or Putatively Actionable  
2 = Molecular Oncology Almanac & PHIAL observed Investigate Actionability or Putatively Actionable  

In [22]:
df['value'] = 0

idx_almanac = df['almanac_bin'].fillna('').isin(['Putatively Actionable', 'Investigate Actionability'])
idx_phial = df['phial_bin'].fillna('').isin(['Putatively Actionable', 'Investigate Actionability'])

df['value'] = 0
df.loc[idx_almanac, 'value'] = 1
df.loc[idx_almanac & idx_phial, 'value'] = 2

In [23]:
df['feature_type'] = df['feature_type'].str.replace('Germline', 'Germline variant')

def pivot_features_table(dataframe, cohort):
    subset = dataframe[dataframe['cohort'].eq(cohort)]
    df = (subset
          .sort_values('value', ascending=False)
          .loc[:, ['patient_id', 'feature_type', 'value']]
          .drop_duplicates(['patient_id', 'feature_type'], keep='first')
          .pivot_table(columns='patient_id', index='feature_type', values='value', fill_value=0)
         )
    display_order = ['Somatic variant', 'Somatic copy number', 'Rearrangement', 'Germline variant', 'Aneuploidy', 'Tumor mutational burden', 'Mutational signature']
    df = df.reindex(display_order)
    return df.T.sort_values(by=df.index.tolist(), axis=0, ascending=False).T

vanallen_pivoted = pivot_features_table(df, 'MEL')
robinson_pivoted = pivot_features_table(df, 'SU2C')
perry_pivoted = pivot_features_table(df, 'OS')
tcga_pivoted = pivot_features_table(df, 'KIRP')

In [24]:
perry_pivoted.to_csv('retrospective.pivoted.perry.txt', sep='\t')
robinson_pivoted.to_csv('retrospective.pivoted.robinson.txt', sep='\t')
tcga_pivoted.to_csv('retrospective.pivoted.tcga.txt', sep='\t')
vanallen_pivoted.to_csv('retrospective.pivoted.vanallen.txt', sep='\t')

## Fraction of samples with therapeutic sensitivity by evidence

In [25]:
def return_max_per_patient(dataframe, samples):
    inverse_map = {
        0.0: 'No sensitive',
        1.0: 'Inferential', 2.0: 'Preclinical', 3.0: 'Clinical evidence', 4.0: 'Clinical trial',
        5.0: 'Guideline', 6.0: 'FDA-Approved'}
    series = pd.Series(inverse_map[0.0], index=samples, name='max_imp')
    grouped = dataframe.groupby(['patient_id'])['sensitive_score']
    for group, value in grouped:
        series.loc[group] = inverse_map[value.max()]
    return series

values_map = {'Inferential': 1.0, 
              'Preclinical': 2.0, 
              'Clinical evidence': 3.0,
              'Clinical trial': 4.0,
              'Guideline': 5.0,
              'FDA-Approved': 6.0}

sensitive = df[df['sensitive_predictive_implication'].notnull()].reset_index(drop=True)
sensitive['sensitive_score'] = sensitive['sensitive_predictive_implication'].replace(values_map)

perry_samples = df[df['cohort'].eq('OS')]['patient_id'].drop_duplicates().tolist()
robinson_samples = df[df['cohort'].eq('SU2C')]['patient_id'].drop_duplicates().tolist()
tcga_samples = df[df['cohort'].eq('KIRP')]['patient_id'].drop_duplicates().tolist()
vanallen_samples = df[df['cohort'].eq('MEL')]['patient_id'].drop_duplicates().tolist()

perry_max = return_max_per_patient(sensitive[sensitive['cohort'].eq('OS')], perry_samples)
robinson_max = return_max_per_patient(sensitive[sensitive['cohort'].eq('SU2C')], robinson_samples)
tcga_max = return_max_per_patient(sensitive[sensitive['cohort'].eq('KIRP')], tcga_samples)
vanallen_max = return_max_per_patient(sensitive[sensitive['cohort'].eq('MEL')], vanallen_samples)

perry_max_vc = perry_max.value_counts().reindex(['FDA-Approved', 'Guideline', 
                                                       'Clinical trial', 'Clinical evidence', 
                                                       'Preclinical', 'Inferential']).fillna(0.0)
perry_max_vc['WT'] = len(perry_samples) - perry_max_vc.astype(int).sum()
perry_max_vc_fract = (round(perry_max_vc / len(perry_samples), 2)*100).astype(int).astype(str)

robinson_max_vc = robinson_max.value_counts().reindex(['FDA-Approved', 'Guideline', 
                                                       'Clinical trial', 'Clinical evidence', 
                                                       'Preclinical', 'Inferential']).fillna(0.0)
robinson_max_vc['WT'] = len(robinson_samples) - robinson_max_vc.astype(int).sum()
robinson_max_vc_fract = (round(robinson_max_vc / len(robinson_samples), 2)*100).astype(int).astype(str)

tcga_max_vc = tcga_max.value_counts().reindex(['FDA-Approved', 'Guideline', 
                                                       'Clinical trial', 'Clinical evidence', 
                                                       'Preclinical', 'Inferential']).fillna(0.0)
tcga_max_vc['WT'] = len(tcga_samples) - tcga_max_vc.astype(int).sum()
tcga_max_vc_fract = (round(tcga_max_vc / len(tcga_samples), 2)*100).astype(int).astype(str)

vanallen_max_vc = vanallen_max.value_counts().reindex(['FDA-Approved', 'Guideline', 
                                                       'Clinical trial', 'Clinical evidence', 
                                                       'Preclinical', 'Inferential']).fillna(0.0)
vanallen_max_vc['WT'] = len(vanallen_samples) - vanallen_max_vc.astype(int).sum()
vanallen_max_vc_fract = (round(vanallen_max_vc / len(vanallen_samples), 2)*100).astype(int).astype(str)

perry_counts_sens = pd.concat([perry_max_vc, perry_max_vc_fract], axis=1).reset_index()
perry_counts_sens.columns = ['evidence', 'counts', 'fraction']

robinson_counts_sens = pd.concat([robinson_max_vc, robinson_max_vc_fract], axis=1).reset_index()
robinson_counts_sens.columns = ['evidence', 'counts', 'fraction']

tcga_counts_sens = pd.concat([tcga_max_vc, tcga_max_vc_fract], axis=1).reset_index()
tcga_counts_sens.columns = ['evidence', 'counts', 'fraction']

vanallen_counts_sens = pd.concat([vanallen_max_vc, vanallen_max_vc_fract], axis=1).reset_index()
vanallen_counts_sens.columns = ['evidence', 'counts', 'fraction']

perry_counts_sens['fraction'] = perry_counts_sens['fraction'].astype(int).divide(100)
robinson_counts_sens['fraction'] = robinson_counts_sens['fraction'].astype(int).divide(100)
tcga_counts_sens['fraction'] = tcga_counts_sens['fraction'].astype(int).divide(100)
vanallen_counts_sens['fraction'] = vanallen_counts_sens['fraction'].astype(int).divide(100)

labels = ['FDA', 'Guideline', 'Clinical trial', 
          'Clinical evidence', 'Preclinical', 'Inferential', 
          'No event associated with therapeutic sensitivity']

perry_counts_sens['label'] = labels
robinson_counts_sens['label'] = labels
tcga_counts_sens['label'] = labels
vanallen_counts_sens['label'] = labels

# Due to rounding, the fraction doesn't quite equal 100, we make a couple of adjustments for the display
#perry_counts_sens.loc[6, 'fraction'] = 0.37
#robinson_counts_sens.loc[6, 'fraction'] = 0.07
tcga_counts_sens.loc[6, 'fraction'] = 0.17
vanallen_counts_sens.loc[6, 'fraction'] = 0.03

perry_counts_sens.to_csv('retrospective.sensitivity-by-evidence.perry.txt', sep='\t', index=False)
robinson_counts_sens.to_csv('retrospective.sensitivity-by-evidence.robinson.txt', sep='\t', index=False)
tcga_counts_sens.to_csv('retrospective.sensitivity-by-evidence.tcga.txt', sep='\t', index=False)
vanallen_counts_sens.to_csv('retrospective.sensitivity-by-evidence.vanallen.txt', sep='\t', index=False)

## Count number of features called between PHIAL/TARGET and MOAlmanac for MEL and Prostate
Per feature type, we count the number of events per cohort and per methodology

In [26]:
feature_types = ['Somatic variant', 'Somatic copy number', 'Rearrangement', 'Germline variant', 
                 'Tumor mutational burden', 'Mutational signature', 'Aneuploidy']
index_values = ['PHIAL/TARGET', 'Molecular Oncology Almanac']

idx_mel = df['cohort'].eq('MEL')
idx_su2c = df['cohort'].eq('SU2C')
idx_os = df['cohort'].eq('OS')
idx_kirp = df['cohort'].eq('KIRP')
idx_almanac = df['almanac_bin'].fillna('').isin(['Putatively Actionable', 'Investigate Actionability'])
idx_phial = df['phial_bin'].fillna('').isin(['Putatively Actionable', 'Investigate Actionability'])

counts_kirp = pd.DataFrame(0, columns = feature_types, index = index_values).T
counts_kirp.loc[feature_types, 'PHIAL/TARGET'] = df[idx_kirp & idx_phial]['feature_type'].value_counts()
counts_kirp.loc[feature_types, 'Molecular Oncology Almanac'] = df[idx_kirp & idx_almanac]['feature_type'].value_counts()
counts_kirp.fillna(0, inplace=True)
counts_kirp = counts_kirp.T

counts_mel = pd.DataFrame(0, columns = feature_types, index = index_values).T
counts_mel.loc[feature_types, 'PHIAL/TARGET'] = df[idx_mel & idx_phial]['feature_type'].value_counts()
counts_mel.loc[feature_types, 'Molecular Oncology Almanac'] = df[idx_mel & idx_almanac]['feature_type'].value_counts()
counts_mel.fillna(0, inplace=True)
counts_mel = counts_mel.T

counts_os = pd.DataFrame(0, columns = feature_types, index = index_values).T
counts_os.loc[feature_types, 'PHIAL/TARGET'] = df[idx_os & idx_phial]['feature_type'].value_counts()
counts_os.loc[feature_types, 'Molecular Oncology Almanac'] = df[idx_os & idx_almanac]['feature_type'].value_counts()
counts_os.fillna(0, inplace=True)
counts_os = counts_os.T

counts_su2c = pd.DataFrame(0, columns = feature_types, index = index_values).T
counts_su2c.loc[feature_types, 'PHIAL/TARGET'] = df[idx_su2c & idx_phial]['feature_type'].value_counts()
counts_su2c.loc[feature_types, 'Molecular Oncology Almanac'] = df[idx_su2c & idx_almanac]['feature_type'].value_counts()
counts_su2c.fillna(0, inplace=True)
counts_su2c = counts_su2c.T

counts_kirp.to_csv('retrospective.feature-type-counts.tcga.txt', sep='\t', index_label='method')
counts_mel.to_csv('retrospective.feature-type-counts.vanallen.txt', sep='\t', index_label='method')
counts_su2c.to_csv('retrospective.feature-type-counts.robinson.txt', sep='\t', index_label='method')
counts_os.to_csv('retrospective.feature-type-counts.perry.txt', sep='\t', index_label='method')

In [27]:
counts_mel

,Somatic variant,Somatic copy number,Rearrangement,Germline variant,Tumor mutational burden,Mutational signature,Aneuploidy
PHIAL/TARGET,167.0,127.0,0.0,0.0,0.0,0.0,0.0
Molecular Oncology Almanac,601.0,137.0,19.0,0.0,44.0,5.0,62.0


In [28]:
counts_su2c

,Somatic variant,Somatic copy number,Rearrangement,Germline variant,Tumor mutational burden,Mutational signature,Aneuploidy
PHIAL/TARGET,108.0,218.0,0.0,0.0,0.0,0.0,0.0
Molecular Oncology Almanac,235.0,227.0,108.0,0.0,4.0,35.0,75.0


In [29]:
counts_os

,Somatic variant,Somatic copy number,Rearrangement,Germline variant,Tumor mutational burden,Mutational signature,Aneuploidy
PHIAL/TARGET,16.0,61.0,0.0,0.0,0.0,0.0,0.0
Molecular Oncology Almanac,27.0,59.0,6.0,0.0,0.0,21.0,33.0


In [30]:
counts_kirp

,Somatic variant,Somatic copy number,Rearrangement,Germline variant,Tumor mutational burden,Mutational signature,Aneuploidy
PHIAL/TARGET,19.0,22.0,0.0,0.0,0.0,0.0,0.0
Molecular Oncology Almanac,72.0,87.0,4.0,0.0,0.0,57.0,10.0


## Count feature types by cohort, evidence, and feature type
Multi-indexed: cohort, assertion type, actionability, feature type

In [31]:
groups = []
for ctype in ['sensitive', 'resistance', 'prognostic']:
    idx_bin = df['almanac_bin'].isin(['Putatively Actionable', 'Investigate Actionability'])
    idx_ctype = ~df[f'{ctype}_predictive_implication'].isnull()
    tmp = df.loc[idx_bin & idx_ctype, :]
    
    for label, group in tmp.groupby(['cohort', 'almanac_bin', f'{ctype}_predictive_implication', 'feature_type']):
        series = pd.Series(label)
        series.loc[4] = ctype
        series.loc[5] = group.shape[0]
        series.loc[6] = group['patient_id'].drop_duplicates().shape[0]
        groups.append(series)
        
counts = pd.concat(groups, axis=1).T
counts.columns = ['cohort', 'bin', 'evidence', 'feature_type', 'assertion', 'event counts', 'patient counts']

(counts
 .loc[:, ['cohort', 'bin', 'assertion', 'evidence', 'event counts', 'patient counts']]
 .to_csv('retrospective.counts-by-category.txt', sep='\t', index=False)
)

In [32]:
df['feature_type'].value_counts()

Somatic variant            1686
Somatic copy number        1240
Mutational signature        620
Rearrangement               255
Aneuploidy                  180
Tumor mutational burden      48
Name: feature_type, dtype: int64

In [33]:
empty_rows = []
for cohort in df['cohort'].unique().tolist():
    for bin_value in ['Putatively Actionable', 'Investigate Actionability']:
        for ctype in ['sensitive', 'resistance', 'prognostic']:
            for evidence in ['FDA-Approved', 'Guideline', 'Clinical trial', 'Clinical evidence', 'Preclinical', 'Inferential']:
                for ftype in ['Somatic variant', 'Somatic copy number', 'Rearrangement', 'Germline variant', 'Mutational signature', 'Tumor mutational burden', 'Aneuploidy']:
                    idx = (counts['cohort'].eq(cohort) 
                       & counts['bin'].eq(bin_value) 
                       & counts['assertion'].eq(ctype) 
                       & counts['evidence'].eq(evidence)
                       & counts['feature_type'].eq(ftype)
                      )
                    if counts[idx].shape[0] == 0:
                        dictionary = {
                            'cohort': cohort, 
                            'bin': bin_value, 
                            'assertion': ctype,
                            'evidence': evidence,
                            'feature_type': ftype,
                            'event counts': 0,
                            'patient counts': 0}
                        empty_rows.append(pd.DataFrame(dictionary, index=[0]))
                    
counts_with_zeros = (pd
                     .concat([counts, pd.concat(empty_rows)])
                     .sort_values(['cohort', 'assertion', 'bin', 'evidence', 'feature_type', 'event counts', 'patient counts'])
                    )
counts_with_zeros.to_csv('retrospective.counts-by-category.txt', sep='\t', index=False)

